# Mini Project
# Part 2: Classification using attribute dataset but not adjacent dataset
Author: Zhicong Sun  
SID: 12032471  
Date: 2021.5.6
## 1 Read dataset and assign feature names to dataset

In [1]:
# import basic module
import numpy as np
import pandas as pd

In [2]:
# read data from txt file, and then assign a feature name to each column of data set, test set and label
attr_data = pd.read_csv("attr.csv",header = None,usecols = [1,2,3,4,5,6])
feature_names = []
for i in range(attr_data.shape[1]):
    feature_names.append("feature" + str(i+1))
attr_data = pd.read_csv("attr.csv",header = None,usecols = [1,2,3,4,5,6],names=feature_names)
train_data = pd.read_csv("attr.csv",header = None,usecols = [1,2,3,4,5,6],nrows=4000,names=feature_names)
train_label = pd.read_csv("label_train.csv",header=None,usecols = [1],names=["label"])
test_data = pd.read_csv("attr.csv",header = None,usecols = [1,2,3,4,5,6],skiprows = 4000,nrows=1298,names=feature_names)
test_label = pd.read_csv("label_test.csv",header=None,usecols = [1],names=["label"])


print("feature names: ",feature_names,"\n")
print("\ndata.head:\n",attr_data.head(),"\n")
print("train data: \n",train_data.head(),"\n")
print("shape of train data: ",train_data.shape,"\n")
print("train label:\n ",train_label.head(),"\n")
print("shape of train label: ",train_label.shape,"\n")
print("test data:\n ",test_data.head(),"\n")
print("shape of test data: ",test_data.shape,"\n")
print("test label: \n",test_label.head(),"\n")
print("shape of test label: ",test_label.shape,"\n")


feature names:  ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6'] 


data.head:
    feature1  feature2  feature3  feature4  feature5  feature6
0         1         2         8        22         0     23574
1         2         2        15        14       229         0
2         1         1         3         0       237      9976
3         2         2         3         0       230     19776
4         1         2         6         0       229     17097 

train data: 
    feature1  feature2  feature3  feature4  feature5  feature6
0         1         2         8        22         0     23574
1         2         2        15        14       229         0
2         1         1         3         0       237      9976
3         2         2         3         0       230     19776
4         1         2         6         0       229     17097 

shape of train data:  (4000, 6) 

train label:
     label
0   2009
1   2004
2   2008
3   2002
4   2007 

shape of train label:  (4000,

## 2 Divide features into categorical value and continous value


In [3]:
# divide features into categorical value and continous value
categorical_val = []
continous_val = []
for column in train_data.columns:
    print('---------------------------')
    print(f"{column} : {train_data[column].unique()}")
    if len(train_data[column].unique()) <= 10:
        categorical_val.append(column)
    else:
        continous_val.append(column)
print("\ncategorical features:",categorical_val)
print("\ncontinous_val features:",continous_val)

---------------------------
feature1 : [1 2 5 3 6 4]
---------------------------
feature2 : [2 1 0]
---------------------------
feature3 : [ 8 15  3  6  7 10  0 14  2 18 16  1  9 13 12 29 39  5 30 27  4 17 33 24
 25 31 44 22 11 28 34 32 26 41 20 43 21 23 37 35 42 19 40]
---------------------------
feature4 : [22 14  0 10  6  7 15 24 28  9 30  3 25 23 18 29  4 16 36 41 26  8 13  5
 17 12  1  2 31 27 34 11 19 33 37 21 43 35 20 39 44 32 40]
---------------------------
feature5 : [  0 229 237 230 286 268 283 236 227 228 274 275 256 231 255 258 278 241
 251 261 240 233 276 254 246 265 247 232 285 244 264 253 242 272 250 234
 270 257 260 259 245 249 267 248 235 262 280 284 266 243 263 288 282 269
 271 252 239 279 290 281 287 289 273 238]
---------------------------
feature6 : [23574     0  9976 ... 53700  9884 53767]

categorical features: ['feature1', 'feature2']

continous_val features: ['feature3', 'feature4', 'feature5', 'feature6']


## 3 Dummy coding of categorical features

In [4]:
# dummy coding of categorical features
total_data = pd.concat([train_data,test_data],axis = 0)
total_dataset = pd.get_dummies(total_data, columns = categorical_val)
print(total_dataset.shape)
train_dataset = total_dataset[0:4000][:]
print(train_dataset.shape)
test_dataset = total_dataset[4000:][:]
print(test_dataset.shape)

(5298, 13)
(4000, 13)
(1298, 13)


## 4 Features scaling of continous features

In [5]:
# standardization
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
data_aft_stdsscsl = train_dataset
data_aft_stdsscsl[continous_val] = standard_scaler.fit_transform(train_dataset[continous_val])
data_aft_stdsscsl.head()

,feature3,feature4,feature5,feature6,feature1_1,feature1_2,feature1_3,feature1_4,feature1_5,feature1_6,feature2_0,feature2_1,feature2_2
0,-0.138214,1.416387,-2.405891,0.269229,1,0,0,0,0,0,0,0,1
1,0.782334,0.629505,0.247000,-1.002596,0,1,0,0,0,0,0,0,1
2,-0.795748,-0.747538,0.339677,-0.464387,1,0,0,0,0,0,0,1,0
3,-0.795748,-0.747538,0.258584,0.064326,0,1,0,0,0,0,0,0,1
4,-0.401227,-0.747538,0.247000,-0.080207,1,0,0,0,0,0,0,0,1


## 5 The Model selction
### 5.1 K Nearest Neighbor(KNN)

In [6]:
from sklearn.metrics import accuracy_score,roc_curve,auc
from sklearn.model_selection import cross_val_score,ShuffleSplit,cross_validate
train_label = train_label.values.ravel()

In [7]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = KNeighborsClassifier()
cv = ShuffleSplit(n_splits=10, test_size=0.4, random_state=0)
scores = cross_validate(clf_knn,data_aft_stdsscsl,train_label,cv=cv,return_train_score=True)
print("Train accuracy of KNN: {:.2f}%".format(scores['train_score'].mean()*100))
print("Test accuracy of KNN: {:.2f}%".format(scores['test_score'].mean()*100))
compare_models = pd.DataFrame(data=[["KNN", scores['train_score'].mean()*100, scores['test_score'].mean()*100]],columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
compare_models

Train accuracy of KNN: 52.25%
Test accuracy of KNN: 29.99%


,Model,Training Accuracy %,Testing Accuracy %
0,KNN,52.245833,29.9875


### 5.2 Support Vector Machine (SVM)

In [8]:
from sklearn.svm import SVC
clf_svm = SVC(random_state = 1)
cv = ShuffleSplit(n_splits=10, test_size=0.4, random_state=0)
scores = cross_validate(clf_svm,data_aft_stdsscsl,train_label,cv=cv,return_train_score=True)
print("Train accuracy of SVM: {:.2f}%".format(scores['train_score'].mean()*100))
print("Test accuracy of SVM: {:.2f}%".format(scores['test_score'].mean()*100))
compare_model_1 = pd.DataFrame(data=[["SVM", scores['train_score'].mean()*100, scores['test_score'].mean()*100]],columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
compare_models = compare_models.append(compare_model_1,ignore_index=True)
compare_models

Train accuracy of SVM: 39.11%
Test accuracy of SVM: 31.67%


,Model,Training Accuracy %,Testing Accuracy %
0,KNN,52.245833,29.98750
1,SVM,39.112500,31.66875


### 5.3 Decision Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier
clf_dtc = DecisionTreeClassifier()
cv = ShuffleSplit(n_splits=10, test_size=0.4, random_state=0)
scores = cross_validate(clf_dtc,data_aft_stdsscsl,train_label,cv=cv,return_train_score=True)
print("Train accuracy of Decision Tree: {:.2f}%".format(scores['train_score'].mean()*100))
print("Test accuracy of Decision Tree: {:.2f}%".format(scores['test_score'].mean()*100))
compare_model_2 = pd.DataFrame(data=[["Decision Tree", scores['train_score'].mean()*100, scores['test_score'].mean()*100]],columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
compare_models = compare_models.append(compare_model_2,ignore_index=True)
compare_models

Train accuracy of Decision Tree: 98.80%
Test accuracy of Decision Tree: 28.66%


,Model,Training Accuracy %,Testing Accuracy %
0,KNN,52.245833,29.98750
1,SVM,39.112500,31.66875
2,Decision Tree,98.804167,28.65625


### 5.4 Neural Netwrok

In [10]:
from sklearn.neural_network import MLPClassifier
clf_neural_net = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(10,10,10), random_state=1,max_iter=10000)
cv = ShuffleSplit(n_splits=10, test_size=0.4, random_state=0)
scores = cross_validate(clf_neural_net,data_aft_stdsscsl,train_label.ravel(),cv=cv,return_train_score=True)
print("Train accuracy of Neural Network: {:.2f}%".format(scores['train_score'].mean()*100))
print("Test accuracy of Neural Network: {:.2f}%".format(scores['test_score'].mean()*100))
compare_model_3 = pd.DataFrame(data=[["Neural Network", scores['train_score'].mean()*100, scores['test_score'].mean()*100]],columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
compare_models = compare_models.append(compare_model_3,ignore_index=True)
compare_models

Train accuracy of Neural Network: 34.74%
Test accuracy of Neural Network: 31.88%


,Model,Training Accuracy %,Testing Accuracy %
0,KNN,52.245833,29.98750
1,SVM,39.112500,31.66875
2,Decision Tree,98.804167,28.65625
3,Neural Network,34.737500,31.88125


### 5.5 Ensembel Model 

In [11]:
from sklearn.ensemble import VotingClassifier
clf_ensembel = VotingClassifier(estimators=[('knn',clf_knn), ('svm',clf_svm), ('decision tree',clf_dtc),('neural network',clf_neural_net)],voting='hard')
cv = ShuffleSplit(n_splits=10, test_size=0.4, random_state=0)
scores = cross_validate(clf_ensembel,data_aft_stdsscsl,train_label,cv=cv,return_train_score=True)
print("Train accuracy of Ensembel Model: {:.2f}%".format(scores['train_score'].mean()*100))
print("Test accuracy of Ensembel Model: {:.2f}%".format(scores['test_score'].mean()*100))
compare_model_4 = pd.DataFrame(data=[["Ensembel Model", scores['train_score'].mean()*100, scores['test_score'].mean()*100]],columns=['Model', 'Training Accuracy %', 'Testing Accuracy %'])
compare_models = compare_models.append(compare_model_4,ignore_index=True)
compare_models

Train accuracy of Ensembel Model: 63.80%
Test accuracy of Ensembel Model: 31.90%


,Model,Training Accuracy %,Testing Accuracy %
0,KNN,52.245833,29.98750
1,SVM,39.112500,31.66875
2,Decision Tree,98.804167,28.65625
3,Neural Network,34.737500,31.88125
4,Ensembel Model,63.804167,31.90000


***
**Note:**
After comparing these models, in can be concluded that the ensembel model is the better one, and I choose it to predict the test label.
***

## 6 Final training and prediction  
***
**Note:**
Now all the train data can be used to train the ensembel model, and the trained model will be used to predict the test data.
***

In [12]:
# features scaling of continous features using standardization
standard_scaler = StandardScaler()
test_data_aft_stdsscsl = test_dataset
test_data_aft_stdsscsl[continous_val] = standard_scaler.fit_transform(test_dataset[continous_val])
test_data_aft_stdsscsl.head()

,feature3,feature4,feature5,feature6,feature1_1,feature1_2,feature1_3,feature1_4,feature1_5,feature1_6,feature2_0,feature2_1,feature2_2
0,-0.284773,1.567937,0.687503,-0.860418,0,1,0,0,0,0,0,1,0
1,-0.150409,-0.726406,0.299379,1.783446,1,0,0,0,0,0,0,0,1
2,0.924504,-0.726406,0.605173,-0.797336,1,0,0,0,0,0,0,0,1
3,-0.419138,-0.726406,0.828638,-1.001792,0,0,0,0,1,0,0,1,0
4,-0.822230,-0.726406,0.863922,-0.756631,0,0,0,0,1,0,0,0,1


In [13]:
# training and prediction
clf_ensembel = VotingClassifier(estimators=[('knn',clf_knn), ('svm',clf_svm), ('decision tree',clf_dtc),('neural network',clf_neural_net)],voting='hard')
clf_ensembel.fit(data_aft_stdsscsl,train_label)
predict_label = clf_ensembel.predict(test_data_aft_stdsscsl)
tmp_test_label = test_label.values
changed_test_label = np.zeros(tmp_test_label.size)
for i in range(tmp_test_label.size):
    changed_test_label[i] = int(tmp_test_label[i][0])
# print(changed_test_label)
print("predict labels: ", predict_label)
accuracy = float((predict_label == changed_test_label).astype(int).sum()) / float(changed_test_label.size)
print("Accuracy on test set: ",accuracy*100,"%")
np.savetxt("predict_testlabel_without_adjlist.txt",predict_label)

predict labels:  [2005 2008 2006 ... 2007 2006 2004]
Accuracy on test set:  31.124807395993837 %


## 7 Disscuss the performance of this classifier


In [14]:
from sklearn.metrics import precision_score,f1_score
micro_precision = precision_score(changed_test_label, predict_label, average="micro")
macro_precision = precision_score(changed_test_label, predict_label, average="macro")
weighted_precision = precision_score(changed_test_label, predict_label, average="weighted")
micro_f1_score = f1_score(changed_test_label, predict_label, average='micro')
print("micro_precision:", micro_precision)
print("macro_precision:", macro_precision)
print("weighted_precision:", weighted_precision)
print("micro_f1_score:", micro_f1_score)

micro_precision: 0.31124807395993837
macro_precision: 0.10281796495576465
weighted_precision: 0.2964209748846245
micro_f1_score: 0.31124807395993837
